
<br>
===========<br>
Inpainting<br>
===========<br>
Inpainting [1]_ is the process of reconstructing lost or deteriorated<br>
parts of images and videos.<br>
The reconstruction is supposed to be performed in fully automatic way by<br>
exploiting the information presented in non-damaged regions.<br>
In this example, we show how the masked pixels get inpainted by<br>
inpainting algorithm based on 'biharmonic equation'-assumption [2]_ [3]_ [4]_.<br>
.. [1]  Wikipedia. Inpainting<br>
        https://en.wikipedia.org/wiki/Inpainting<br>
.. [2]  Wikipedia. Biharmonic equation<br>
        https://en.wikipedia.org/wiki/Biharmonic_equation<br>
.. [3]  S.B.Damelin and N.S.Hoang. "On Surface Completion and Image<br>
        Inpainting by Biharmonic Functions: Numerical Aspects",<br>
        International Journal of Mathematics and Mathematical Sciences,<br>
        Vol. 2018, Article ID 3950312<br>
        :DOI:`10.1155/2018/3950312`<br>
.. [4]  C. K. Chui and H. N. Mhaskar, MRA Contextual-Recovery Extension of<br>
        Smooth Functions on Manifolds, Appl. and Comp. Harmonic Anal.,<br>
        28 (2010), 104-113,<br>
        :DOI:`10.1016/j.acha.2009.04.004`<br>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from skimage import data
from skimage.morphology import disk, binary_dilation
from skimage.restoration import inpaint

In [ ]:
image_orig = data.astronaut()

Create mask with six block defect regions

In [ ]:
mask = np.zeros(image_orig.shape[:-1], dtype=bool)
mask[20:60, 0:20] = 1
mask[160:180, 70:155] = 1
mask[30:60, 170:195] = 1
mask[-60:-30, 170:195] = 1
mask[-180:-160, 70:155] = 1
mask[-60:-20, 0:20] = 1

add a few long, narrow defects

In [ ]:
mask[200:205, -200:] = 1
mask[150:255, 20:23] = 1
mask[365:368, 60:130] = 1

add randomly positioned small point-like defects

In [ ]:
rstate = np.random.RandomState(0)
for radius in [0, 2, 4]:
    # larger defects are less common
    thresh = 3 + 0.25 * radius  # make larger defects less commmon
    tmp_mask = rstate.randn(*image_orig.shape[:-1]) > thresh
    if radius > 0:
        tmp_mask = binary_dilation(tmp_mask, disk(radius, dtype=bool))
    mask[tmp_mask] = 1

Apply defect mask to the image over the same region in each color channel

In [ ]:
image_defect = image_orig * ~mask[..., np.newaxis]

In [ ]:
image_result = inpaint.inpaint_biharmonic(image_defect, mask, channel_axis=-1)

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2)
ax = axes.ravel()

In [ ]:
ax[0].set_title('Original image')
ax[0].imshow(image_orig)

In [ ]:
ax[1].set_title('Mask')
ax[1].imshow(mask, cmap=plt.cm.gray)

In [ ]:
ax[2].set_title('Defected image')
ax[2].imshow(image_defect)

In [ ]:
ax[3].set_title('Inpainted image')
ax[3].imshow(image_result)

In [ ]:
for a in ax:
    a.axis('off')

In [ ]:
fig.tight_layout()
plt.show()